### Paxillin movie segmentation and object feature analysis

with static images

Liya Ding, 2024.04

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import umap
from joblib import dump, load
import os.path

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.transforms import Bbox

import tifffile 
import czifile

import skimage
from skimage.morphology import remove_small_objects
from skimage.measure import label, regionprops, regionprops_table
from skimage.color import label2rgb

from scipy import ndimage
from scipy.ndimage import distance_transform_cdt
from scipy.ndimage import gaussian_filter

from sklearn.decomposition import PCA
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# function from aicssegmentation tool
from utils.vessel_2d import filament_2d_wrapper
from utils.pre_processing_utils import intensity_normalization


In [2]:
czimovie_dir =  '/mnt/d/lding/FA/data/FHL2-KI-U2OS-Actin405-Pax568-endoFHL2488_annabel_20240226'

csv_output_dir = '/mnt/d/lding/FA/analysis_results/pax_3D/pax_3D_0226/Vess-LocThr-GlThr-Open_tab10_csv'
plot_output_dir = '/mnt/d/lding/FA/analysis_results/pax_3D/pax_3D_0226/Vess-LocThr-GlThr-Open_tab10_plot'
seg_output_dir = '/mnt/d/lding/FA/analysis_results/pax_3D/pax_3D_0226/Vess-LocThr-GlThr-Open_tab10_seg'

pixel_size = 0.0706
time_point = 0

if not os.path.isdir(csv_output_dir):
    os.makedirs(csv_output_dir)
if not os.path.isdir(plot_output_dir):
    os.makedirs(plot_output_dir)
if not os.path.isdir(seg_output_dir):
    os.makedirs(seg_output_dir)

In [3]:
# build the colormap with iterative tab10
tab10 = cm.get_cmap('tab10', 10)
for i in range(10):
    tab10.colors = np.concatenate([tab10.colors,tab10.colors],axis=0)    
tab10.colors = np.concatenate([np.zeros([1,4]),tab10.colors],axis=0)

newmap = cm.get_cmap('tab10', 4000+1)
newmap.colors = tab10.colors[0:4000+1,:]
newmap.colors[0,:]  = np.zeros([1,4])
newmap.colors[0,3] = 1

### for each movie run through all the time points

In [4]:
# flag for some optinal parts
flag_plot = True
flag_plot_save = True
flag_seg_save = True
flag_csv_save = True
flag_run_all = True
flag_panel_plot = True

In [5]:
prop_df_pax_all=pd.read_csv(os.path.join('/mnt/d/lding/FA/analysis_results/static_image_csv/','static_images_cluster_labels_allcells_nc'+str(7)+'.csv'))

        

In [6]:
### for each movie, make sub-folders for different output for easier viewing in imagej
movie_mask_output_dir = os.path.join(seg_output_dir,  'mask')
movie_seg_output_dir = os.path.join(seg_output_dir,  'seg')
movie_label_output_dir = os.path.join(seg_output_dir,  'label')
movie_labeltoggle_output_dir = os.path.join(seg_output_dir,  'togglelabel')
movie_rgb_output_dir = os.path.join(seg_output_dir,  'rgb')    
movie_contour_output_dir = os.path.join(plot_output_dir,  'contour')
movie_quiver_cell_output_dir = os.path.join(plot_output_dir,  'quiver_cell')
movie_quiver_obj_output_dir = os.path.join(plot_output_dir,  'quiver_obj')
movie_label_color_output_dir = os.path.join(plot_output_dir,  'label_color')
movie_rgb_plot_output_dir = os.path.join(plot_output_dir,  'rgb_plot')
movie_panels_plot_output_dir = os.path.join(plot_output_dir,  'panel_plot')
movie_cluster_plot_output_dir = os.path.join(plot_output_dir,  'cluster_tab10_minibatc_1P_plot')
if not os.path.isdir(movie_cluster_plot_output_dir):   os.makedirs(movie_cluster_plot_output_dir)

In [9]:
filenames = [x for x in os.listdir(czimovie_dir) if os.path.isfile(os.path.join(czimovie_dir, x)) and ('.czi' in x)]

# if not in run all mode, only test with the first image of the first movie
if not flag_run_all:
    filenames = filenames[0:1]

for filenameID in range(len(filenames)):
    filename = filenames[filenameID]
    img = czifile.imread(os.path.join(czimovie_dir,filename))
    pax_img = img[0,0,0,:,:,:,0].squeeze()
    intensity_scaling_param = [10,40]
    norm_pax_img = intensity_normalization(pax_img, scaling_param=intensity_scaling_param)
    MIP_pax_img = norm_pax_img.max(axis=0)
    label_pax_seg = tifffile.imread(os.path.join(movie_label_output_dir, 'SS_pax_seglabel_'+filename+'_MIP'+'.tif'))   

    max_label = label_pax_seg.max()+1

    
    # print(newmap.colors[:,0:3])
    less_col_prop_df_pax_all = prop_df_pax_all[['filename','cell_ID','label','cluster_ID_MiniBatc']]
    cluster_label_pax_seg = np.zeros_like(label_pax_seg)
    this_cell_table = less_col_prop_df_pax_all[less_col_prop_df_pax_all['filename'] == filename]
    
    for iL in range(1,label_pax_seg.max()+1):  
        # print(iL)
        cluster_ID = this_cell_table[this_cell_table['label']==iL]
        if(cluster_ID.shape[0]==1):
            cluster_label_pax_seg[label_pax_seg==iL] = cluster_ID.iloc[0,-1]+1

    if flag_plot:

        if flag_panel_plot:
            # plot the main outputs together for quick viewing
            
            fig = plt.figure(figsize=(12,12), dpi=512)
            # ax[0].imshow(MIP_pax_img, cmap=plt.cm.gray)
            # ax[0].axis('off')

            # ax[1].imshow(label_pax_seg, cmap=newmap, vmax = 4001,vmin = 0, interpolation='none')
            # ax[1].axis('off')

            plt.imshow(cluster_label_pax_seg, cmap=newmap,vmax = 4001,vmin = 0, interpolation='none')
            plt.axis('off')
            
            tifffile.imsave(os.path.join(movie_cluster_plot_output_dir, 'label_clusters_'+filename+'_MIP'+'.tif'),cluster_label_pax_seg.astype(np.uint16)) 

            if flag_plot_save: 
                # save the plots and subplots for easier viewing
                plt.savefig(os.path.join(movie_cluster_plot_output_dir,'VB_clusters_'+str(filenameID)+'.png'))
            
            if flag_run_all:
                plt.close(fig) 
                    

                

In [33]:
this_cell_table

,filename,cell_ID,label,cluster_ID_MiniBatc
0,MAX_C2-Cell-01.tif,0,1,3
1,MAX_C2-Cell-01.tif,0,2,3
2,MAX_C2-Cell-01.tif,0,3,0
3,MAX_C2-Cell-01.tif,0,4,3
4,MAX_C2-Cell-01.tif,0,5,3
...,...,...,...,...
49165,Control 1.czi,0,2314,4
49166,Control 1.czi,0,2315,4
49167,Control 1.czi,0,2316,4
49168,Control 1.czi,0,2317,5


In [28]:
newmap.colors

array([[0.        , 0.        , 0.        , 1.        ],
       [0.12156863, 0.46666667, 0.70588235, 1.        ],
       [1.        , 0.49803922, 0.05490196, 1.        ],
       ...,
       [0.49803922, 0.49803922, 0.49803922, 1.        ],
       [0.7372549 , 0.74117647, 0.13333333, 1.        ],
       [0.09019608, 0.74509804, 0.81176471, 1.        ]])

In [29]:
newmap.to_numpy

AttributeError: 'ListedColormap' object has no attribute 'to_numpy'

In [ ]:
timeID

In [ ]:
cluster_ID = this_cell_time[this_cell_time['label']==iL]


In [ ]:
iL

In [ ]:
this_cell_time

In [ ]:
cluster_ID.shape[0]


In [ ]:
iL

In [ ]:
this_cell_table['label']

In [ ]:
cluster_ID['cluster_ID_MeanShif']

In [ ]:
label_pax_seg.max()

In [ ]:
'd='+f'{cell_edge_dist:.0f}'+',oo='+f'{regionprops_pax[iL].orientation:.1f}'+',co='+f'{cell_edge_orient:.1f}'+',diff='+f'{diff_orient:.1f}'

In [ ]:
f'{cell_edge_dist:.0f}'

### belows are for debugging, keep it here until no longer needed

In [ ]:
pax_image_label_overlay = label2rgb(label_pax_seg, image=smooth_pax_img, bg_label=0,kind='overlay',alpha=0.5)
plt.rcParams["figure.figsize"] = (15,15)
plt.imshow(pax_image_label_overlay)

In [ ]:
plt.rcParams["figure.figsize"] = (15,15)
plt.imshow(pax_image_label_overlay, cmap=plt.cm.RdBu_r,vmax=0.1,vmin=0)
# plt.quiver(obj_Y,obj_X, cell_U,cell_V,color='blue')
Q = plt.quiver(obj_Y,obj_X, -obj_U,obj_V,color='m')#width=0.005,headwidth=0.1,headaxislength=0.02,headlength=0.01)
# plt.quiver(obj_Y,obj_X, cell_U,cell_V,color='blue')

plt.axis('off')

In [ ]:
plt.imshow(distance_taxicab)

grid_mask = np.zeros_like(new_cell_mask)
grid_mask[::30,::30] = new_cell_mask[::30,::30]
to_plot_X = X[grid_mask>0]
to_plot_Y = Y[grid_mask>0]
to_plot_U = n_v[grid_mask>0]
to_plot_H = n_h[grid_mask>0]

# plt.contour(X, Y, distance_taxicab,6,linewidths=2)    
plt.quiver(to_plot_X,to_plot_Y, -to_plot_U,to_plot_H,color='blue')
plt.axis('off')